### Some analysis

In [2]:
from officelib.xllib import *
from officelib.const import xlconst as xlc
import os
import re

In [3]:
keys = {
    'temperature.pv',
    'agitation.pv',
    'ph.outputDown',
    'ph.manDown',
    'do.manUp',
    'do.outputDown',
    'do.pv',
    'do.manDown',
    'do.outputUp',
    'maingas.man',
    'maingas.mode',
    'maingas.pv',
    'MFCs.air',
    'MFCs.n2',
    'MFCs.o2',
    'MFCs.co2',
    'level.pv',
    'Elapsed',
    'Elapsed.hr',
    'Time'
}
def hide_columns(ws):
    for col in ws.UsedRange.Columns:
        c = col.Cells(1,1)
        if c.Value not in keys:
            col.Hidden=True

In [4]:
def data_range(ws, name, begin_offset=0):
    c1 = ws.Cells.Find(name)
    if c1 is None: raise ValueError(name)
    c1 = c1.Offset(2+begin_offset,1)
    c2 = c1.End(xlc.xlDown)
    return c1, c2

class SeriesProxy():
    def __init__(self, ws, name, xname="Elapsed", yax=1, begin_offset=0):
        self.ws = ws
        self.name = name
        self.yax = yax
        self.s = None
        self.xname = xname
        self.begin_offset=begin_offset
        
    def create(self, chart):
        e1, e2 = data_range(self.ws, self.xname, self.begin_offset)
        c1, c2 = data_range(self.ws, self.name, self.begin_offset)
        cr = self.ws.Cells.Range
        self.xr = cr(e1, e2)
        self.yr = cr(c1, c2)
        self.s = CreateDataSeries(chart, self.xr, self.yr, self.name)
        
    def set_name(self, n):
        self.s.Name = n 
        
class ChartProxy():
    def __init__(self, ws, chart=None, *args):
        self.ws = ws
        self.chart = chart or CreateChart(ws, *args)
        self.chart.ChartStyle = 240  # Excel 2017 style
        self.series = []
        
    def make_series(self, series):
        for s in series:
            s.create(self.chart)
            self.series.append(s)
            
    def _axlabel(self, ax, t):
        ax.HasTitle=True
        ax.AxisTitle.Text=t
    
    def xlabel(self, t):
        xax = self.chart.Axes(xlc.xlCategory, xlc.xlPrimary)
        self._axlabel(xax, t)
        
    def ylabel(self, t):
        yax = self.chart.Axes(xlc.xlValue, xlc.xlPrimary)
        self._axlabel(yax, t)
        
    def ylabel2(self, t):
        yax = self.chart.Axes(xlc.xlValue, xlc.xlSecondary)
        self._axlabel(yax, t)
        
    def title(self, t):
        self.chart.HasTitle = True
        self.chart.ChartTitle.Text = t
        
    def copyto(self, r):
        # chart parent -> ChartObject
        self.chart.Parent.Copy()
        r.Paste()
         
def _listify(args):
    l = []
    if isinstance(args, str):
        return [args]
    for a in args:
        if isinstance(a, (list,tuple)):
            l.extend(_listify(a))
        else:
            l.append(a)
    return l
    
def make_chart(names, title=None, xlabel=None, ylabel=None, xname="Elapsed", *, uws=None, begin_offset=0):
    uws = uws or ws
    c = ChartProxy(uws)
        
    names = _listify(names)
    c.make_series(SeriesProxy(uws,n, xname, 1, begin_offset) for n in names)
    
    # optional setup
    if title: c.title(title)
    if xlabel: c.xlabel(xlabel)
    if ylabel: c.ylabel(ylabel)
    
    return c

### Example code for now to use functions

In [5]:
# # MFCs
# mfcc = [
#     "MFCs.Air",
#     "MFCs.O2",
#     "MFCs.N2",
#     "MFCs.CO2"
# ]
# make_chart(mfcc, "MFC Flow", "Time(s)", "Flow Rate (LPM)")
# make_chart("do.pv", "DO PV", "Time(s)", "DO PV (%)")

#### Some analysis

In [6]:
def column_data(name):
    c1 = ws.Cells.Find(name)
    c2 = c1.End(xlc.xlDown)
    data = ws.Cells.Range(c1.Offset(2,1),c2).Value
    return [d[0] for d in data]
    
def add_af1(name, new, form):
    e1 = ws.Cells.Find(name)
    e1.Offset(1,2).EntireColumn.Insert()
    e1.Offset(1,2).Value = new
    
    c1 = e1.Offset(2,2)
    c2 = e1.End(xlc.xlDown).Offset(1,2)
    c1.Value = form % e1.Offset(2,1).GetAddress(0,0)
    afr = ws.Cells.Range(c1, c2)
    c1.AutoFill(afr)
    return c1, c2
    

def nch(ws, n):
    c = CreateChart(ws)
    c.Location(xlc.xlLocationAsNewSheet, n)
    return ws.Parent.Charts(n)

def linest(s):
    c = ws.Cells(1,1).End(xlc.xlToRight).Offset(1, 3)
    c.Value = "kLa"
    c2 = c.Offset(2,1)
    c2.Value = "=Index(Linest(%s,%s), 1)" % (s.yr.Address, s.xr.Address)
    return c2

def mkcmp():
    global nwb, nws, cmpc
    nwb = xl.Workbooks.Add()
    nws = nwb.Worksheets(1)
    cmpc = nch(nws, "Comparison")
    c1 = nws.Cells.Range("B2")
    c1.Value = "Name"
    c1.Offset(1,2).Value = "LPM"
    c1.Offset(1,3).Value = "kla.val"
    nws.Cells.Range(c1.Offset(0, 1), c1.Offset(0, 2)).Merge()
    c1.Offset(0, 1).Value = "Compiled kLA Data"
        
    
def lincmp(name, kla):
    c1 = nws.Cells.Range("B3")
    if not c1.Value:
        c = c1
    elif c1.Value and not c1.Offset(2,1).Value:
        c = c1.Offset(2,1)
    else:
        c = c1.End(xlc.xlDown).Offset(2,1)
    c.Value = name
    c.Offset(1,2).Value = re.match(r".*(\d+\.\d+)LPM", name).groups()[0]
    c.Offset(1,3).Value = kla
        
    
def gfp(f): return os.path.abspath(f)

def check_no_flow(name, nmax=2):
    down = column_data(name)
    ndown = len([d for d in down if d > 0])
    if ndown > nmax:
        print("ERROR: Check %s for %r"%(name, f))
        
def check_flow(name, exp, nmax=5):
    flow = column_data(name)
    nflow = len([d for d in flow if round(d, 0) != round(exp, 0)])
    if nflow > nmax:
        print("ERROR: Check %s for current file"%(name))

    
def sorteddir():
    return sorted((f for f in os.listdir() if os.path.isfile(f)))

def trendclr(c):
    ns = c.SeriesCollection().Count
    l = c.Legend.LegendEntries
    for i in range(2*ns, ns, -1):
        l(i).Delete()
        
def mktrend(c):
    AddTrendlines(c)
    trendclr(c)
    
    
def anylf(wb,ws,name,min_start=0,rampdown=False):
    e1, e2 = add_af1("Elapsed", "Elapsed.hr", "=%s/3600")
    ws.Cells.Range(e1, e2).NumberFormat = "0.000"
    
    # calculate time offset
    tdat = column_data('Elapsed.hr')
    offs = next((i for i, t in enumerate(tdat) if t > min_start), 0)
    
    if rampdown:
        cname = "Ramp Down"
        target = "do.pv"
        yax = "DO PV (%)"
    else:
        cname = "Ramp Up"
        target = "-do.ln(pv)"
        yax = "-LN(100-DOPV)"
        add_af1("do.pv", target, "=-ln(100-%s)")

    c = make_chart(target, "%s %s"%(name, cname), "Time(hr)", 
                   yax, "Elapsed.hr", uws=ws, begin_offset=offs)
    c.series[0].set_name(name)
    mktrend(c.chart)

    if rampdown:
        # by the time 4 values are recorded, maingas should be set to the test's sp....
        cd = column_data('maingas.man')
        if len(cd) < 4:
            mg = cd[-1]
        else:
            mg = cd[3]
        check_flow("ph.outputDown", 0.1/mg*100)
        kla=0
    else:
        check_no_flow("ph.outputDown")
        check_no_flow("do.outputDown")
        check_flow("do.outputUp", 100)  # flow is always 100 for Sam's tests
        
        # MFCs
        mfcc = [
            "MFCs.Air",
            "MFCs.O2",
            "MFCs.N2",
            "MFCs.CO2"
        ]
        make_chart(mfcc, "MFC Flow", "Time(s)", "Flow Rate (LPM)")
        make_chart("do.pv", "DO PV", "Time(s)", "DO PV (%)")
        
        lc = linest(c.series[0])
        kla = lc.Value
        
    return c, kla
    
def analyze3(files, min_start=0.0, cmp_name="compiled"):
    # globals make debugging easy
    global nwb, nws, nc, wb, ws, e1, e2, pc, c, cmpc
    mkcmp()
    xl.DisplayAlerts=False
    for f in files:
        print("Analyzing", f)
        _, _, _, ramp, *name, _ = f.split()
        name = ' '.join(name)
        rampdown = ramp == 'rampdown'
        wb = xl.Workbooks.Open(gfp(f))
        ws = wb.Worksheets(1)
        
        c, kla = anylf(wb,ws,name,min_start,rampdown)
             
        if not rampdown:
            c.copyto(cmpc)
            lincmp(name, kla)
            
        wb.SaveAs(gfp(".\\analyzed\\"+f.replace(".csv", ".xlsx")), FileFormat=xlc.xlOpenXMLWorkbook)
        wb.Close(True)
        
    make_chart("kla.val", "kLa", "Main Gas", "kLa", "LPM", uws=nws)
        
    nwb.SaveAs(gfp(".\\analyzed\\%s.xlsx"%cmp_name))
    print("Finished analysis!")

In [64]:
fp = 'C:\\Users\\natha\\Documents\\test\\Sam Microsparger Test\\L Plate C 0.1-0.2 overnight 170817'
xl = Excel()
os.chdir(fp)
os.makedirs("analyzed", exist_ok=True)
files = sorteddir()
with screen_lock(xl):
    for wb in xl.Workbooks: wb.Close(False)
    analyze3(files, 0.2, "L Place C data")

Analyzing DO micro test rampdown L plate A 0.100LPM 1 170817.csv
Analyzing DO micro test rampdown L plate A 0.100LPM 2 170818.csv
Analyzing DO micro test rampdown L plate A 0.100LPM 3 170818.csv
Analyzing DO micro test rampdown L plate A 0.125LPM 1 170817.csv
Analyzing DO micro test rampdown L plate A 0.125LPM 2 170818.csv
Analyzing DO micro test rampdown L plate A 0.125LPM 3 170818.csv
Analyzing DO micro test rampdown L plate A 0.150LPM 1 170817.csv
Analyzing DO micro test rampdown L plate A 0.150LPM 2 170818.csv
Analyzing DO micro test rampdown L plate A 0.150LPM 3 170818.csv
Analyzing DO micro test rampdown L plate A 0.175LPM 1 170817.csv
Analyzing DO micro test rampdown L plate A 0.175LPM 2 170818.csv
Analyzing DO micro test rampdown L plate A 0.175LPM 3 170819.csv
Analyzing DO micro test rampdown L plate A 0.200LPM 1 170818.csv
Analyzing DO micro test rampdown L plate A 0.200LPM 2 170818.csv
Analyzing DO micro test rampdown L plate A 0.200LPM 3 170819.csv
Analyzing DO micro test r

AttributeError: 'NoneType' object has no attribute 'groups'

In [41]:
fp = 'C:\\Users\\natha\\Documents\\test\\Sam Microsparger Test'
xl = Excel()
os.chdir(fp)
os.makedirs("analyzed", exist_ok=True)
files = ["DO micro test rampup L plate C 0.2LPM 1 170817.csv"]
with screen_lock(xl):
    for wb in xl.Workbooks: wb.Close(False)
    analyze3(files, 0.2, "out")

Analyzing DO micro test rampup L plate C 0.2LPM 1 170817.csv


com_error: (-2147352567, 'Exception occurred.', (0, 'Microsoft Excel', "Sorry, we couldn't find C:\\Users\\natha\\Documents\\test\\Sam Microsparger Test\\DO micro test rampup L plate C 0.2LPM 1 170817.csv. Is it possible it was moved, renamed or deleted?", 'xlmain11.chm', 0, -2146827284), None)

In [122]:
fp = 'C:\\Users\\natha\\Documents\\test\\Sam Microsparger Test\\L Plate D 0.1-0.2 overnight 170831'
xl = Excel()
os.chdir(fp)
os.makedirs("analyzed", exist_ok=True)
files = [f for f in sorteddir() if "%" not in f and "rampup" in f]
with screen_lock(xl):
    for wb in xl.Workbooks: wb.Close(False)
    analyze3(files, 0.2, "L Plate D data")

Analyzing DO micro test rampup L plate D 0.100LPM 1 170901.csv
Analyzing DO micro test rampup L plate D 0.100LPM 1 170904.csv
Analyzing DO micro test rampup L plate D 0.100LPM 2 170901.csv
Analyzing DO micro test rampup L plate D 0.100LPM 2 170904.csv
Analyzing DO micro test rampup L plate D 0.100LPM 3 170901.csv
Analyzing DO micro test rampup L plate D 0.100LPM 3 170904.csv
Analyzing DO micro test rampup L plate D 0.125LPM 1 170901.csv
Analyzing DO micro test rampup L plate D 0.125LPM 1 170904.csv
Analyzing DO micro test rampup L plate D 0.125LPM 2 170901.csv
Analyzing DO micro test rampup L plate D 0.125LPM 2 170904.csv
Analyzing DO micro test rampup L plate D 0.125LPM 3 170901.csv
Analyzing DO micro test rampup L plate D 0.125LPM 3 170904.csv
Analyzing DO micro test rampup L plate D 0.150LPM 1 170901.csv
Analyzing DO micro test rampup L plate D 0.150LPM 1 170904.csv
Analyzing DO micro test rampup L plate D 0.150LPM 2 170901.csv
Analyzing DO micro test rampup L plate D 0.150LPM 2 170

In [121]:
fp = 'C:\\Users\\natha\\Documents\\test\\Sam Microsparger Test\\L Plate D 2mm 0.1-0.2 overnight 170831'
xl = Excel()
os.chdir(fp)
os.makedirs("analyzed", exist_ok=True)
files = [f for f in sorteddir() if "%" not in f and "rampup" in f]
with screen_lock(xl):
    for wb in xl.Workbooks: wb.Close(False)
    analyze3(files, 0.2, "L Plate D 2mm data")

Analyzing DO micro test rampup L plate D 2mm 0.100LPM 1 170905.csv
Analyzing DO micro test rampup L plate D 2mm 0.100LPM 2 170906.csv
Analyzing DO micro test rampup L plate D 2mm 0.100LPM 3 170906.csv
Analyzing DO micro test rampup L plate D 2mm 0.125LPM 1 170905.csv
Analyzing DO micro test rampup L plate D 2mm 0.125LPM 2 170906.csv
Analyzing DO micro test rampup L plate D 2mm 0.125LPM 3 170906.csv
Analyzing DO micro test rampup L plate D 2mm 0.150LPM 1 170906.csv
Analyzing DO micro test rampup L plate D 2mm 0.150LPM 2 170906.csv
Analyzing DO micro test rampup L plate D 2mm 0.150LPM 3 170906.csv
Analyzing DO micro test rampup L plate D 2mm 0.175LPM 1 170906.csv
Analyzing DO micro test rampup L plate D 2mm 0.175LPM 2 170906.csv
Analyzing DO micro test rampup L plate D 2mm 0.175LPM 3 170906.csv
Analyzing DO micro test rampup L plate D 2mm 0.200LPM 1 170906.csv
Analyzing DO micro test rampup L plate D 2mm 0.200LPM 2 170906.csv
Analyzing DO micro test rampup L plate D 2mm 0.200LPM 3 170906

In [17]:
fp = 'C:\\Users\\natha\\Documents\\test\\Sam Microsparger Test\\L Plate E 4mm 0.1-0.2 overnight 170915'
xl = Excel()
os.chdir(fp)
os.makedirs("analyzed", exist_ok=True)
files = [f for f in sorteddir() if "%" not in f and "rampup" in f]
with screen_lock(xl):
    for wb in xl.Workbooks: wb.Close(False)
    analyze3(files, 0.2, "L Plate E 4mm data")

Analyzing DO micro test rampup L plate E 4mm 0.100LPM 1 170916.csv
Analyzing DO micro test rampup L plate E 4mm 0.100LPM 2 170916.csv
Analyzing DO micro test rampup L plate E 4mm 0.100LPM 3 170916.csv
Analyzing DO micro test rampup L plate E 4mm 0.125LPM 1 170916.csv
Analyzing DO micro test rampup L plate E 4mm 0.125LPM 2 170916.csv
Analyzing DO micro test rampup L plate E 4mm 0.125LPM 3 170916.csv
Analyzing DO micro test rampup L plate E 4mm 0.150LPM 1 170916.csv
Analyzing DO micro test rampup L plate E 4mm 0.150LPM 2 170916.csv
Analyzing DO micro test rampup L plate E 4mm 0.150LPM 3 170917.csv
Analyzing DO micro test rampup L plate E 4mm 0.175LPM 1 170916.csv
Analyzing DO micro test rampup L plate E 4mm 0.175LPM 2 170916.csv
Analyzing DO micro test rampup L plate E 4mm 0.175LPM 3 170917.csv
Analyzing DO micro test rampup L plate E 4mm 0.200LPM 1 170916.csv
Analyzing DO micro test rampup L plate E 4mm 0.200LPM 2 170916.csv
Analyzing DO micro test rampup L plate E 4mm 0.200LPM 3 170917